#### Домашнее задание.
## Урок 10 - Библиотека Request и REST API
#### Выполнил работу: Данил Кузнецов
Вам будут даны файлы с событиями:

Установки:
ID пользователя,
IP-адрес установки.
Платежа:
ID пользователя,
сумма платежа.

Нужно будет посчитать средний чек пользователя из разных стран (средний чек — ARPU).


Бонусное (на оценку не влияет): попробуйте распределить страны по 3-м категориям (на tier-1, tier-2, tier-3), на основании размера среднего чека от пользователя из их страны.


(В методичке расписан API запрос к серверу, который возвращает страну в ответе при API запросе).

In [1]:
import pandas as pd
import numpy as np
import requests
import json
import time

In [2]:
payments = pd.read_csv('les10_payments.csv')
installs = pd.read_csv('les10_installs.csv')
installs

,event_time,event_date,user_id,ip,device_type
0,2021-01-01 03:11:29,2021-01-01,216149,96.28.104.236,iPhone X
1,2021-01-01 03:13:38,2021-01-01,216151,96.28.48.22,iPhone 11
2,2021-01-01 03:33:58,2021-01-01,216152,74.56.1.222,iPad 5
3,2021-01-01 03:47:43,2021-01-01,216153,70.93.185.219,IPod touch 7
4,2021-01-01 03:58:57,2021-01-01,214545,174.62.104.183,iPhone XS
...,...,...,...,...,...
995,2021-01-05 17:49:17,2021-01-05,131706,176.59.50.238,xiaomi-Redmi Note 7
996,2021-01-05 17:51:07,2021-01-05,216756,46.211.46.213,Xiaomi-Redmi 5A
997,2021-01-05 17:53:50,2021-01-05,106302,88.155.87.158,Xiaomi-Redmi 7A
998,2021-01-05 18:03:26,2021-01-05,118279,46.211.46.213,Xiaomi-Redmi 5A


In [3]:
URL = 'http://api.sypexgeo.net/json/{}'
ip_list = installs['ip']

country_list = {}

for ip in ip_list:
    try:
        json_text = requests.get(URL.format(ip)).text
        result = json.loads(json_text)
        country_list[ip] = result['country']['name_ru']                         
    except:
        time.sleep(1)
country_list

{'96.28.104.236': 'США',
 '96.28.48.22': 'США',
 '74.56.1.222': 'Канада',
 '70.93.185.219': 'США',
 '174.62.104.183': 'США',
 '136.152.143.53': 'США',
 '107.77.204.66': 'США',
 '172.56.23.177': 'США',
 '69.57.248.48': '',
 '94.25.227.184': 'Россия',
 '94.25.179.24': 'Россия',
 '93.170.247.18': 'Россия',
 '71.15.206.127': 'США',
 '99.61.135.143': 'США',
 '223.104.145.68': 'Китай',
 '198.27.189.54': 'США',
 '94.245.180.131': 'Россия',
 '176.64.20.103': 'Казахстан',
 '75.169.146.24': 'США',
 '74.132.178.36': 'США',
 '173.28.51.231': 'США',
 '75.21.229.82': 'США',
 '172.58.31.31': 'США',
 '97.100.152.88': 'США',
 '89.42.205.105': 'Казахстан',
 '68.10.251.87': 'США',
 '85.249.24.81': 'Россия',
 '72.140.212.215': 'Канада',
 '217.76.13.209': 'Армения',
 '79.170.138.60': 'Россия',
 '81.222.185.238': 'Россия',
 '172.125.169.36': 'США',
 '123.19.97.75': 'Вьетнам',
 '188.232.208.186': 'Россия',
 '106.220.108.54': 'Индия',
 '73.220.83.204': 'США',
 '104.12.39.222': 'США',
 '75.155.125.241': 'Канад

In [4]:
type(country_list)

dict

## На этом моменте я потратил около суток времени) Очень важный вопрос: Почему тип у нас - словарь, а превращаем в таблицу с помощью json_normalize ? При этом класическое pd.DataFrame не работает?? 

In [5]:
data = country_list

In [6]:
data = pd.json_normalize(country_list)

In [7]:
country_ip = data.T.reset_index()
country_ip.columns = ['ip','country']

In [8]:
country_ip

,ip,country
0,96.28.104.236,США
1,96.28.48.22,США
2,74.56.1.222,Канада
3,70.93.185.219,США
4,174.62.104.183,США
...,...,...
818,84.54.44.99,Россия
819,85.140.22.30,Россия
820,31.47.171.53,Россия
821,176.59.50.238,Россия


In [9]:
final_table = installs.merge(country_ip, on='ip', how='left')


In [10]:
final_table['country'].unique()

array(['США', 'Канада', '', 'Россия', 'Китай', 'Казахстан', 'Армения',
       'Вьетнам', 'Индия', 'Узбекистан', 'Беларусь', 'Украина', 'Молдова',
       'Кыргызстан', 'Мьянма', 'Боливия', 'Литва', 'Филиппины',
       'Аргентина', 'Латвия', 'Словения', 'Бразилия', 'Южная Корея',
       'Великобритания', 'Индонезия', nan, 'Эстония', 'Тринидад и Тобаго',
       'Польша', 'Пуэрто-Рико', 'Франция', 'Палестина', 'Таджикистан',
       'Ирак', 'Румыния', 'Грузия', 'Саудовская Аравия'], dtype=object)

### у нас ненайденные страны по разному обозначены, ниже решил их обозначить одним способом

In [11]:
final_table.fillna('Undefinded')

,event_time,event_date,user_id,ip,device_type,country
0,2021-01-01 03:11:29,2021-01-01,216149,96.28.104.236,iPhone X,США
1,2021-01-01 03:13:38,2021-01-01,216151,96.28.48.22,iPhone 11,США
2,2021-01-01 03:33:58,2021-01-01,216152,74.56.1.222,iPad 5,Канада
3,2021-01-01 03:47:43,2021-01-01,216153,70.93.185.219,IPod touch 7,США
4,2021-01-01 03:58:57,2021-01-01,214545,174.62.104.183,iPhone XS,США
...,...,...,...,...,...,...
995,2021-01-05 17:49:17,2021-01-05,131706,176.59.50.238,xiaomi-Redmi Note 7,Россия
996,2021-01-05 17:51:07,2021-01-05,216756,46.211.46.213,Xiaomi-Redmi 5A,Украина
997,2021-01-05 17:53:50,2021-01-05,106302,88.155.87.158,Xiaomi-Redmi 7A,Украина
998,2021-01-05 18:03:26,2021-01-05,118279,46.211.46.213,Xiaomi-Redmi 5A,Украина


In [12]:
for i in range(len(final_table['country'])) :
    if final_table['country'][i] == '' :
        final_table['country'][i] ='Undefinded'
     

C:\Users\4dani\AppData\Local\Temp/ipykernel_9640/1808093948.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  final_table['country'][i] ='Undefinded'


In [13]:
final_table['country'].unique()

array(['США', 'Канада', 'Undefinded', 'Россия', 'Китай', 'Казахстан',
       'Армения', 'Вьетнам', 'Индия', 'Узбекистан', 'Беларусь', 'Украина',
       'Молдова', 'Кыргызстан', 'Мьянма', 'Боливия', 'Литва', 'Филиппины',
       'Аргентина', 'Латвия', 'Словения', 'Бразилия', 'Южная Корея',
       'Великобритания', 'Индонезия', nan, 'Эстония', 'Тринидад и Тобаго',
       'Польша', 'Пуэрто-Рико', 'Франция', 'Палестина', 'Таджикистан',
       'Ирак', 'Румыния', 'Грузия', 'Саудовская Аравия'], dtype=object)

# Почему то функция .fillna() не работает и нан все равно остатется в таблице... будет здорово если расскажешь почему, а то я не смог понять

### Осталось заджойнить таблицы и сгруппировать.

In [16]:
final_table = final_table.merge(payments, on='user_id', how='left')

In [19]:
final_table = final_table.groupby(['country']).agg({'user_id':'nunique','revenue_usd':'sum'})

In [20]:
final_table['ARPU'] = final_table['revenue_usd']/final_table['user_id']
final_table

,user_id,revenue_usd,ARPU
country,,,
Undefinded,1,0.000000,0.000000
Аргентина,1,0.000000,0.000000
Армения,1,25.650000,25.650000
Беларусь,15,8.500000,0.566667
Боливия,1,0.000000,0.000000
Бразилия,1,0.000000,0.000000
Великобритания,1,0.000000,0.000000
Вьетнам,6,0.000000,0.000000
Грузия,2,0.000000,0.000000


### Теперь добавлю тир-лист

In [24]:
def tier_rang(APRU):
    if APRU > 10:
        return 'tier-1'
    if 1 <= APRU <= 10:
        return 'tier-2'
    if APRU < 1:
        return 'tier-3'
    
final_table['tier_list'] = final_table['ARPU'].apply(tier_rang)

In [25]:
final_table.columns = ['count_users','revenue','ARPU','tier_list']

final_table.sort_values('tier_list')

,count_users,revenue,ARPU,tier_list
country,,,,
Армения,1,25.650000,25.650000,tier-1
США,228,4520.610000,19.827237,tier-1
Филиппины,1,1.000000,1.000000,tier-2
Украина,97,567.858678,5.854213,tier-2
Россия,371,600.947011,1.619803,tier-2
Казахстан,32,97.895554,3.059236,tier-2
Undefinded,1,0.000000,0.000000,tier-3
Польша,2,0.000000,0.000000,tier-3
Пуэрто-Рико,1,0.000000,0.000000,tier-3
